In [1]:
# Tensorly library
import tensorly as tl
import numpy as np
from functools import reduce

In [103]:
"""
TENSOR TO TENSOR REGRESSION v1    
  Inputs:
    X - Input Tensor, shape (N, I_1, ..., I_{M1})
    Y - Output Tensor, shape (N, J_1, ..., J_{M2})
    rank - The rank of the solution weights matrix
    lambda_reg - how much l2 regularization to use
    return_factors - whether to include the weight matrix factors along with the weight matrix itself
    eps - the precision of our estimate
    
  Outputs:
    W - weight matrix tensor, shape (I_1, ..., I_{M1}, J_1, ..., J_{M2})
    factors - List of CP factors which forms the CP decomposition of W. Will only be returned if return_factors==True
"""
def tensor_to_tensor_regression_2(X, Y, rank, lambda_reg=0.0, return_factors=False, eps=1e-6):
    # Check that the N is consistent between X and Y tensors
    if X.shape[0] != Y.shape[0]:
        print("Wrong leading dimensions for tensors X and Y")
    
    # Number of examples (leading dimension in the tensor)
    N = X.shape[0]
    M1 = len(X.shape[1:])
    M2 = len(Y.shape[1:])
    
    # Setup the sizes of the X and Y matrices
    I = reduce(lambda x, y: x * y, X.shape[1:])
    J = reduce(lambda x, y: x * y, Y.shape[1:])
    
    # Initialize the u_r and v_r vectors which we will directly optimize
    Us = [np.random.random((X.shape[m1 + 1], rank))-0.5 for m1 in range(M1)]
    Vs = [np.random.random((Y.shape[m2 + 1], rank))-0.5 for m2 in range(M2)]
    
    # Matricize X and Y
#     X_mat = tl.unfold(X, mode=0)
    Y_vec = tl.tensor_to_vec(Y)
    
    steps = 0
    prev_error = -1
    #errors = [] # TODO: Remove for production (only used for testing)
    # Keep optimizing until the error has converged
    while steps < 5:
        
        # Optimizing all the Us
        for m1 in range(M1):
            C_r_mats = []
            for r in range(rank):
                # Columns we will kronecker together
                cols_to_kron = []
                for m1_p in range(M1):
                    if m1_p != m1:
                        # r-th column of U_m1
                        cols_to_kron.append(Us[m1_p][:,r])
                for m2_p in range(M2):
                    cols_to_kron.append(Vs[m2_p][:,r])
                contractor_shape = tuple(x for i, x in enumerate(X.shape[1:]) if i != m1) + Y.shape[1:]
                contractor = tl.tenalg.kronecker(cols_to_kron).reshape(contractor_shape)
                X_modes = [i for i in range(1, M1 + 1) if i != m1 + 1]
                # Create the C_r tensor
                C_r = tl.tenalg.contract(X, X_modes, contractor, range(tl.ndim(X) - 2)) 
                C_r_mats.append(tl.unfold(C_r, 1))
            C_mat_t = tl.concatenate(C_r_mats, axis=0)
            reg_mat = None
            for m1_p in range(M1):
                if m1_p != m1:
                    to_dot = np.matmul(tl.transpose(Us[m1_p]),Us[m1_p]) 
                    if reg_mat is None:
                        reg_mat = to_dot
                    else:
                        reg_mat = tl.dot(reg_mat, to_dot)
            for m2_p in range(M2):
                to_dot = np.matmul(tl.transpose(Us[m1_p]),Us[m1_p]) 
                if reg_mat is None:
                    reg_mat = to_dot
                else:
                    reg_mat = tl.dot(reg_mat, to_dot)
            reg_mat = lambda_reg * tl.tenalg.kronecker([reg_mat, tl.eye(X.shape[m1 + 1])])
            U_vec = np.matmul(np.matmul(np.linalg.inv(np.matmul(C_mat_t, tl.transpose(C_mat_t)) + reg_mat), C_mat_t), Y_vec)
            Us[m1] = tl.vec_to_tensor(U_vec, Us[m1].shape)
                
        # Optimizing all the Vs    
        for m2 in range(M2):
            Y_m2 = tl.unfold(Y, m2 + 1)
            d_r_vecs = []
            for r in range(rank):
                # Columns we will kronecker together
                cols_to_kron = []
                for m1_p in range(M1):
                    # r-th column of U_m1
                    cols_to_kron.append(Us[m1_p][:,r])
                for m2_p in range(M2):
                    if m2 != m2_p:
                        cols_to_kron.append(Vs[m2_p][:,r])
                contractor_shape = X.shape[1:] + tuple(y for i, y in enumerate(Y.shape[1:]) if i != m2)
                contractor = tl.tenalg.kronecker(cols_to_kron).reshape(contractor_shape)
                X_modes = [i for i in range(1, M2 + 1)]
                # Create the C_r tensor
                d_r = tl.tensor_to_vec(tl.tenalg.contract(X, X_modes, contractor, range(tl.ndim(X) - 1)))
                d_r_vecs.append(d_r.reshape((d_r.shape[0], 1)))
            D_mat = tl.concatenate(d_r_vecs, axis=1)
            # Compute the regularization component
            reg_mat = None
            for m1_p in range(M1):
                to_dot = np.matmul(tl.transpose(Us[m1_p]),Us[m1_p]) 
                if reg_mat is None:
                    reg_mat = to_dot
                else:
                    reg_mat = tl.dot(reg_mat, to_dot)
            for m2_p in range(M2):
                if m2_p != m2:
                    to_dot = np.matmul(tl.transpose(Us[m1_p]),Us[m1_p]) 
                    if reg_mat is None:
                        reg_mat = to_dot
                    else:
                        reg_mat = tl.dot(reg_mat, to_dot)
            reg_mat *= lambda_reg
            # Do the final update
            Vs[m2] = np.matmul(np.matmul(Y_m2, D_mat), np.linalg.inv(np.matmul(tl.transpose(D_mat), D_mat) + reg_mat))
        
        # Compute error here
        # Construct W
        weight_shape = X.shape[1:] + Y.shape[1:]
        W = tl.zeros(weight_shape)
        for r in range(rank):
            to_kron = []
            for m1 in range(M1):
                to_kron.append(Us[m1][:,r])
            for m2 in range(M2):
                to_kron.append(Vs[m2][:,r])
            W += tl.tenalg.kronecker(to_kron).reshape(weight_shape)
        Y_pred = tl.tenalg.contract(X, range(1, M1 + 1), W, range(M1))
        error = np.square(Y_pred - Y).mean() 
        
        # Determine if we have converged
        if prev_error >= 0 and abs(prev_error - error) < eps:
            print("Converged after", steps, "steps. Final Error:", error)
            return W
        else:
            print("Step:", steps, "Error:", error)
        print(error)
        # Reset the previous error
        prev_error = error
        # Next step
        steps += 1
        
    

In [104]:
N = 11
X = np.random.random((N, 5, 7))
Y = np.zeros((N, 2, 3))
# Setup Y tensor with some dummy data
for n in range(N):
    x = X[n]
    Y[n] = np.array([[x[0, 0] + x[1, 1] , 2 * x[1, 0] - x[3, 2], (x[4, 5] + 1) ** 2],
                    [-x[1, 6] + 3 * x[1, 5] ,  - x[0, 6] - x[3, 3], (x[2, 5] + 2) ** 2]])
# Now fit it
W = tensor_to_tensor_regression_2(X, Y, 5, lambda_reg=100)
# We can verify our answer is correct with the following
Y_pred = tl.tenalg.contract(X, range(1, tl.ndim(X)), W, range(tl.ndim(X) - 1))
print(np.square(Y_pred - Y).mean()) # Should print the same as the final error

7.277881273511988
9.721645066975952
9.456567153501327


LinAlgError: Singular matrix

In [70]:
# For timing
import time

start_time = time.time()
W = tensor_to_tensor_regression_2(X, Y, 5, lambda_reg=10)
print(time.time() - start_time)

0.023020267486572266


In [3]:
tuple(x for i, x in enumerate((9,5,3,3,2,1)) if i != 1)

(9, 3, 3, 2, 1)

In [6]:
arr = [i for i in range(5) if i != 2]
print(arr)
arr.pop(1)
print(arr)

[0, 1, 3, 4]
[0, 3, 4]


In [82]:
a = np.random.random((5,5))
b = np.ones((5,5))
c = tl.dot(a, b)
print(c, a)

[[3.44404736 3.44404736 3.44404736 3.44404736 3.44404736]
 [1.78752933 1.78752933 1.78752933 1.78752933 1.78752933]
 [2.52109255 2.52109255 2.52109255 2.52109255 2.52109255]
 [1.12365716 1.12365716 1.12365716 1.12365716 1.12365716]
 [2.44437316 2.44437316 2.44437316 2.44437316 2.44437316]] [[0.80903149 0.79328326 0.69959133 0.18264116 0.95950012]
 [0.26203797 0.00428095 0.17310839 0.56625577 0.78184625]
 [0.91942317 0.15854651 0.92242789 0.22079746 0.29989752]
 [0.21094672 0.06005086 0.00557695 0.17442414 0.6726585 ]
 [0.14708786 0.15139415 0.92567852 0.35986236 0.86035027]]
